<a href="https://colab.research.google.com/github/plk456/strokeprediction/blob/main/strokeprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
n=pd.read_csv('hearthstoke.csv')
print(n.head())

      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


In [3]:
from sklearn.preprocessing import LabelEncoder
le_gender=LabelEncoder()
le_ever_married=LabelEncoder()
le_work_type=LabelEncoder()
le_Residence=LabelEncoder()
le_smoking=LabelEncoder()
n['gender']=le_gender.fit_transform(n['gender'])
n['ever_married']=le_ever_married.fit_transform(n['ever_married'])
n['work_type']=le_work_type.fit_transform(n['work_type'])
n['Residence_type']=le_Residence.fit_transform(n['Residence_type'])
n['smoking_status']=le_smoking.fit_transform(n['smoking_status'])
print(n)

         id  gender   age  hypertension  heart_disease  ever_married  \
0      9046       1  67.0             0              1             1   
1     51676       0  61.0             0              0             1   
2     31112       1  80.0             0              1             1   
3     60182       0  49.0             0              0             1   
4      1665       0  79.0             1              0             1   
...     ...     ...   ...           ...            ...           ...   
5105  18234       0  80.0             1              0             1   
5106  44873       0  81.0             0              0             1   
5107  19723       0  35.0             0              0             1   
5108  37544       1  51.0             0              0             1   
5109  44679       0  44.0             0              0             1   

      work_type  Residence_type  avg_glucose_level   bmi  smoking_status  \
0             2               1             228.69  36.6   

# i am deleting "id" column because it is not useful for me in this dataset

In [4]:
n=n.drop(['id'],axis=1)

In [5]:
print(n)

      gender   age  hypertension  heart_disease  ever_married  work_type  \
0          1  67.0             0              1             1          2   
1          0  61.0             0              0             1          3   
2          1  80.0             0              1             1          2   
3          0  49.0             0              0             1          2   
4          0  79.0             1              0             1          3   
...      ...   ...           ...            ...           ...        ...   
5105       0  80.0             1              0             1          2   
5106       0  81.0             0              0             1          3   
5107       0  35.0             0              0             1          3   
5108       1  51.0             0              0             1          2   
5109       0  44.0             0              0             1          0   

      Residence_type  avg_glucose_level   bmi  smoking_status  stroke  
0              

# Here you can see Nan values

In [6]:
print(n.iloc[:,8:9])

       bmi
0     36.6
1      NaN
2     32.5
3     34.4
4     24.0
...    ...
5105   NaN
5106  40.0
5107  30.6
5108  25.6
5109  26.2

[5110 rows x 1 columns]


In [7]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
imputer.fit(n.iloc[:,8:9])
n.iloc[:,8:9]=imputer.transform(n.iloc[:,8:9])


In [8]:
print(n)

      gender   age  hypertension  heart_disease  ever_married  work_type  \
0          1  67.0             0              1             1          2   
1          0  61.0             0              0             1          3   
2          1  80.0             0              1             1          2   
3          0  49.0             0              0             1          2   
4          0  79.0             1              0             1          3   
...      ...   ...           ...            ...           ...        ...   
5105       0  80.0             1              0             1          2   
5106       0  81.0             0              0             1          3   
5107       0  35.0             0              0             1          3   
5108       1  51.0             0              0             1          2   
5109       0  44.0             0              0             1          0   

      Residence_type  avg_glucose_level        bmi  smoking_status  stroke  
0         

In [49]:
from sklearn.model_selection import train_test_split
x=n.drop(['stroke'],axis=1)
X=x.head(1000)
y=n['stroke']
Y=y.head(1000)


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [50]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler_x_train=scaler.fit_transform(x_train)
scaler_x_test=scaler.fit_transform(x_test)

In [51]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score , confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [35]:
#pip install xgboost

In [52]:
models={
    'logistic':LogisticRegression(),
    'svm':SVC(kernel='rbf'),
    'random':RandomForestClassifier(),
    'Decision':DecisionTreeClassifier(),
    'xgboost':xgb.XGBClassifier()
}
accuracy_scores = dict()
f1_scores = dict()

In [53]:
for model_name,model in models.items():
    model.fit(scaler_x_train,y_train)
    y_pred=model.predict(scaler_x_test)

    accuracy_scores[model_name] = accuracy_score(y_test,y_pred) * 100
    f1_scores[model_name] = f1_score(y_test,y_pred) * 100

In [54]:
print("="*55)
print("|\tModel Name\tAccuracy Score\tF1 Score     |")
print("="*55)
for k in accuracy_scores.keys():
    print(f"|{k:^24}{accuracy_scores[k]:>10.2f}{f1_scores[k]:>12.2f}\t      |")
print("="*55)

|	Model Name	Accuracy Score	F1 Score     |
|        logistic             79.00       50.00	      |
|          svm                78.00       46.34	      |
|         random              77.00       45.24	      |
|        Decision             72.50       45.54	      |
|        xgboost              77.50       45.78	      |


In [59]:
w=[1,67,0,1,1,2, 1,228.69,36,1]
w=np.array(w)
w=w.reshape(1,-1)
w_pred=model.predict(w)
print(w_pred)

[1]
